<a href="https://colab.research.google.com/github/BardRimon/Study/blob/main/Introduction_in_NLP/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%9A%D0%BB%D1%8E%D0%BA%D0%B8%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тема 7. Определение тональности текста







## импорт датасетов

In [1]:
! wget https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv
! wget https://www.dropbox.com/s/gr4z1x39y1j6dtx/rusentiment_test.csv

--2025-06-03 13:26:56--  https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/y17smfk1ptufngw720uny/rusentiment_random_posts.csv?rlkey=p9e77phv8eu6fwh6tou0fz232 [following]
--2025-06-03 13:26:57--  https://www.dropbox.com/scl/fi/y17smfk1ptufngw720uny/rusentiment_random_posts.csv?rlkey=p9e77phv8eu6fwh6tou0fz232
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc54c7fc393f0dbadbb9d1944504.dl.dropboxusercontent.com/cd/0/inline/Cq7p8tudso0A18mOIrDdxLcX7US5x3v_msoH0DO1glJUt4qzuF4VOylWZ0787wi_RnRz0VYxVVpj5ok01LZMlOq4y4vqqI40nveFo27mE7272L6lL2mQtOx82izrJ2JbNUciaf1rpq9Mi1Z7fh1ZZqXI/file# [following]
--2025-06-03 13:26:57--  https://uc54c7fc393f0dbadbb9

In [4]:
# prompt: импортируй файлы .csv и разбей на train and test

import pandas as pd
from sklearn.model_selection import train_test_split

df_train = pd.read_csv('rusentiment_random_posts.csv')
df_test = pd.read_csv('rusentiment_test.csv')

# Assuming your CSV has a column named 'text' for text data and 'label' for sentiment label
X_train = df_train['text']
y_train = df_train['label']

X_test = df_test['text']
y_test = df_test['label']

# If you want to split the training data further into train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
y_train.value_counts()

,count
label,
neutral,8323
positive,4635
skip,3190
speech,2826
negative,2294


## Задача 1: Использовать в классификации внешний словарь тональностей.


In [12]:
!pip install nltk

In [15]:
# prompt: используя большой словарь тональностей реши задачу классификации для текстов на русском языке из интернета, имея следующие метки классов: neutral	8323
# positive	4635
# skip	3190
# speech	2826
# negative	2294

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Simplified external sentiment dictionary (example)
# In a real scenario, you would load a comprehensive dictionary.
sentiment_dict = {
    'хорошо': 'positive',
    'отлично': 'positive',
    'прекрасно': 'positive',
    'плохо': 'negative',
    'ужасно': 'negative',
    'кошмар': 'negative',
    'нейтрально': 'neutral',
    'обычно': 'neutral',
    'нормально': 'neutral',
}

stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'http\S+', '', text) # Remove URLs
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ\s]', '', text) # Remove special characters
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

def get_sentiment_from_dict(text):
    sentiment_scores = {'positive': 0, 'negative': 0, 'neutral': 0}
    words = text.split()
    for word in words:
        if word in sentiment_dict:
            sentiment = sentiment_dict[word]
            sentiment_scores[sentiment] += 1

    # Basic rule: if more positive words than negative, classify as positive, etc.
    if sentiment_scores['positive'] > sentiment_scores['negative']:
        return 'positive'
    elif sentiment_scores['negative'] > sentiment_scores['positive']:
        return 'negative'
    elif sentiment_scores['neutral'] > 0 and sentiment_scores['positive'] == 0 and sentiment_scores['negative'] == 0:
        return 'neutral'
    else:
        return 'skip' # Or another fallback class

# Apply preprocessing
X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)

# Option 1: Use the sentiment dictionary directly (simpler, less accurate)
y_train_dict_pred = X_train_processed.apply(get_sentiment_from_dict)
y_test_dict_pred = X_test_processed.apply(get_sentiment_from_dict)

print("Classification Report using Sentiment Dictionary Directly:")
print(classification_report(y_test, y_test_dict_pred, zero_division=0))


# Option 2: Incorporate sentiment dictionary as a feature (more complex, potentially better)
# This is a basic example of feature engineering with the dictionary.
# You could also use TF-IDF and then add dictionary features.

def add_sentiment_features(text):
    sentiment_scores = {'positive_score': 0, 'negative_score': 0, 'neutral_score': 0}
    words = text.split()
    for word in words:
        if word in sentiment_dict:
            sentiment = sentiment_dict[word]
            if sentiment == 'positive':
                sentiment_scores['positive_score'] += 1
            elif sentiment == 'negative':
                sentiment_scores['negative_score'] += 1
            elif sentiment == 'neutral':
                sentiment_scores['neutral_score'] += 1
    return list(sentiment_scores.values())

train_sentiment_features = pd.DataFrame(list(X_train_processed.apply(add_sentiment_features)),
                                        columns=['positive_score', 'negative_score', 'neutral_score'])
test_sentiment_features = pd.DataFrame(list(X_test_processed.apply(add_sentiment_features)),
                                       columns=['positive_score', 'negative_score', 'neutral_score'])

# Combine sentiment features with TF-IDF features
tfidf = TfidfVectorizer(max_features=5000) # Limit features for simplicity
X_train_tfidf = tfidf.fit_transform(X_train_processed)
X_test_tfidf = tfidf.transform(X_test_processed)

import scipy.sparse as sp
X_train_combined = sp.hstack((X_train_tfidf, sp.csr_matrix(train_sentiment_features.values)))
X_test_combined = sp.hstack((X_test_tfidf, sp.csr_matrix(test_sentiment_features.values)))


# Train a classifier on combined features
model = LogisticRegression(max_iter=1000)
model.fit(X_train_combined, y_train)
y_test_combined_pred = model.predict(X_test_combined)

print("\nClassification Report using TF-IDF and Sentiment Dictionary Features:")
print(classification_report(y_test, y_test_combined_pred, zero_division=0))



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Classification Report using Sentiment Dictionary Directly:
              precision    recall  f1-score   support

    negative       0.35      0.02      0.04       258
     neutral       0.80      0.01      0.01      1420
    positive       0.62      0.01      0.02       536
        skip       0.12      0.99      0.21       346
      speech       0.00      0.00      0.00       407

    accuracy                           0.12      2967
   macro avg       0.38      0.21      0.06      2967
weighted avg       0.54      0.12      0.04      2967


Classification Report using TF-IDF and Sentiment Dictionary Features:
              precision    recall  f1-score   support

    negative       0.47      0.23      0.31       258
     neutral       0.65      0.84      0.73      1420
    positive       0.52      0.46      0.49       536
        skip       0.56      0.25      0.35       346
      speech       0.90      0.83      0.86       407

    accuracy                           0.65      2967
 

## Задача 2: Улучшить качество базовой предсказательной модели на тестовой выборке за счет добавления и модификации признаков.


## Задача 3: Сравнить качество классификации на леммах и подтокенах.


## Задача 4: Обучить fasttext-классификатор, сравнить качество классификации с предобученными эмбеддингами и обученными с нуля при классификации.